# Praca domowa nr 3 bomus
## Piotr Fic

In [1]:
#Pakiety
import pandas as pd
import numpy as np
import sklearn
import category_encoders
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge, LinearRegression, Lasso, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
np.random.seed(123)

# Część z PD2 + niezbędne operacje

### Wczytanie zbioru danych

In [2]:
df = pd.read_csv("allegro-api-transactions.csv")

### Krótkie obejrzenie zbioru

In [3]:
df.head(5)

,lp,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating,it_location,main_category
0,0,2016-04-03 21:21:08,4753602474,"['Komputery', 'Dyski i napędy', 'Nośniki', 'No...",1,1,radzioch666,59.99,1,997,0,50177,Warszawa,Komputery
1,1,2016-04-03 15:35:26,4773181874,"['Odzież, Obuwie, Dodatki', 'Bielizna damska',...",1,1,InwestycjeNET,4.90,1,9288,0,12428,Warszawa,"Odzież, Obuwie, Dodatki"
2,2,2016-04-03 14:14:31,4781627074,"['Dom i Ogród', 'Budownictwo i Akcesoria', 'Śc...",1,1,otostyl_com,109.90,1,895,0,7389,Leszno,Dom i Ogród
3,3,2016-04-03 19:55:44,4783971474,"['Książki i Komiksy', 'Poradniki i albumy', 'Z...",1,1,Matfel1,18.50,0,971,0,15006,Wola Krzysztoporska,Książki i Komiksy
4,4,2016-04-03 18:05:54,4787908274,"['Odzież, Obuwie, Dodatki', 'Ślub i wesele', '...",1,1,PPHU_RICO,19.90,1,950,0,32975,BIAŁYSTOK,"Odzież, Obuwie, Dodatki"


## + rozbicie kolumny 'date'

In [4]:
df['day'] = int(df.date[0][8:10])
df['month'] = int(df.date[0][5:7])
df['year'] = int(df.date[0][0:4])
df['hour'] = int(df.date[0][10:13])
df = df.drop('date', axis=1)

## + usunięcie kolumn 'lp' i 'id'

In [5]:
df = df.drop(['lp', 'item_id'], axis=1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420020 entries, 0 to 420019
Data columns (total 15 columns):
categories                420020 non-null object
pay_option_on_delivery    420020 non-null int64
pay_option_transfer       420020 non-null int64
seller                    420020 non-null object
price                     420020 non-null float64
it_is_allegro_standard    420020 non-null int64
it_quantity               420020 non-null int64
it_is_brand_zone          420020 non-null int64
it_seller_rating          420020 non-null int64
it_location               420020 non-null object
main_category             420020 non-null object
day                       420020 non-null int64
month                     420020 non-null int64
year                      420020 non-null int64
hour                      420020 non-null int64
dtypes: float64(1), int64(10), object(4)
memory usage: 48.1+ MB


In [7]:
#Transformacja string
df['it_location'] = df['it_location'].str.lower()
df['seller'] = df['seller'].str.lower()

## + podział na zbiór treningowy i testowy

In [8]:
X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 123)

## Encoding zmiennych kategorycznych
Muszę zmienić nieco metody na takie, które będą funkcjonować dla zmiennych o dużym zbiorze unikalnych kategorii.

In [9]:
#Kolumny do encodingu
cols = list(X.select_dtypes('object').columns)
cols

['categories', 'seller', 'it_location', 'main_category']

### Target encoding

In [10]:
te = category_encoders.TargetEncoder(cols=cols)
te_train = te.fit_transform(X_train, y_train)
te_test = te.transform(X_test)

### James-Stein encoder

In [11]:
js_en = category_encoders.JamesSteinEncoder(cols = cols)
js_train = js_en.fit_transform(X_train, y_train)
js_test = js_en.transform(X_test)

### Leave-one-out

In [12]:
leo = category_encoders.leave_one_out.LeaveOneOutEncoder(cols=cols)
leo_train = leo.fit_transform(X_train, y_train)
leo_test = leo.transform(X_test)

## Modelowanie

W celu zbadania wpływu encodingu porównam miary osiągnięte przez model lasso regression.

In [13]:
def model(X_train, X_test, y_train, y_test):
    model = Lasso()
    
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, predict))
    r2 = r2_score(y_test, predict)
    
    return(rmse, r2)

In [14]:
te_score = model(te_train, te_test, y_train, y_test)
js_score = model(js_train, js_test, y_train, y_test)
leo_score = model(leo_train, leo_test, y_train, y_test)
result = pd.DataFrame({'target-encoding': te_score,
                      'James-Stein': js_score,
                      'Leave-one-out': leo_score},
                      index=['RMSE', 'R2'])

In [15]:
result

,target-encoding,James-Stein,Leave-one-out
RMSE,538.481645,538.92795,538.379723
R2,0.069394,0.06785,0.069746


Wyniki modelu są bardzo słabe. Określenie przyczyny wymagało by dłuższej analizy i może głębszemu przyjrzeniu się również zbiorowi danych. Zmiana algorytmu z regresji na żaden z innych podstawowych klasyfikatorów nie przyniosła znaczącej poprawy wyników. Ponad to w tym przykładzie każdą z metod encodingu moglibyśmy stosować zamiennie, wyniki w zasadzie nie różnią się względem użytych encodingów. Przyczyną tego może być ich podobieństwo w algorytmach imputacji. 